# Parse and Load .gdf
Parse and the gdf file into 2 csv files, 1 for metadata and the other for relations data. This is for further processing in `R`.

In [5]:
from langdetect import detect

In [4]:
import pandas as pd

Split `.gdf` file into metadata and relation files for processing in `R`. 

In [22]:
def parse_gdf_channel(file_name):
    with open(file_name) as f:
        rows = [line.strip().split(',') for line in f.readlines()]
        # print(rows)
        metadata = [row for row in rows if len(row) > 3]
        graph_data = [row for row in rows if len(row) == 3]
        return pd.DataFrame(metadata[1:], columns=metadata[0]), pd.DataFrame(
            graph_data[1:], columns=graph_data[0])

def parse_gdf_video(file_name):
    with open(file_name, encoding='utf-8') as f:
        rows = [line.strip().split(',') for line in f.readlines()]
        # print(rows)
        metadata = [row[:14] for row in rows if len(row) > 3]
        graph_data = [row for row in rows if len(row) == 3]
        
        return pd.DataFrame(metadata[1:], columns=metadata[0]), pd.DataFrame(
            graph_data[1:], columns=graph_data[0])
    
def all_caps_to_proper(string):
    out_string = ''
    for token in string.split(' '):
        if token.isupper():
            out_string += ' ' + token.capitalize()
        else:
            out_string += ' ' + token
    return out_string.strip()
            

In [57]:
def join_update_clean_dfs(df_graph, df_metadata, title, replace_node_ids=True):
    df_metadata.rename(columns={
        'nodedef>name VARCHAR': 'nodeid',
        'label VARCHAR': 'label',
        'subscriberCount INT': 'subscriberCount'
    },
        inplace=True)
    df_graph.rename(columns={
        'edgedef>node1 VARCHAR': 'nodeid_1',
        'node2 VARCHAR': 'nodeid_2',
        'directed BOOLEAN': 'directed'
    },
        inplace=True)
    if replace_node_ids is True:
        df_graph = pd.merge(df_graph,
                            df_metadata[['label', 'nodeid']],
                            left_on='nodeid_1',
                            right_on='nodeid',
                            how='right')[['label', 'nodeid_2', 'directed'
                                          ]].rename(columns={'label': 'node1'})

        df_graph = pd.merge(df_graph,
                            df_metadata[['label', 'nodeid']],
                            left_on='nodeid_2',
                            right_on='nodeid',
                            how='right')[['node1', 'label', 'directed'
                                          ]].rename(columns={'label': 'node2'})
    else:
        df_graph = pd.merge(df_graph,
                            df_metadata[['label', 'nodeid']],
                            left_on='nodeid_1',
                            right_on='label',
                            how='right')[['label', 'nodeid_2', 'directed'
                                          ]].rename(columns={'label': 'node1'})

        df_graph = pd.merge(df_graph,
                            df_metadata[['label', 'nodeid']],
                            left_on='nodeid_2',
                            right_on='label',
                            how='right')[['node1', 'label', 'directed'
                                          ]].rename(columns={'label': 'node2'})
    return df_metadata, df_graph


def load_video_data(filename):
    df_video_metadata, df_video_rel = parse_gdf_video(filename + '.gdf')
    # write un-cleaned relations to file for use in gephi later
    df_video_rel.to_csv(filename + '_relations_orig.csv')
    # clean up titles for language classification
    df_video_metadata['label_clean'] = df_video_metadata['label VARCHAR'].apply(all_caps_to_proper)
    # detect video title language 
    df_video_metadata['title_language'] = df_video_metadata['label_clean'].apply(detect)
    # clean column names and update node ids
    df_video_metadata, df_video_rel = join_update_clean_dfs(df_video_rel, df_video_metadata, filename)
    df_video_metadata['url'] = 'https://www.youtube.com/watch?v=' + df_video_metadata['nodeid']
    # get channel ids for seed videos
    seed_channel_ids = list(df_video_metadata[df_video_metadata['isSeed VARCHAR'] == 'yes']['channelId VARCHAR'])
    # label whether or not all videos are in the seed channel
    df_video_metadata['seed_channel'] = df_video_metadata['channelId VARCHAR'].apply(lambda x : True if x in seed_channel_ids else False)
    df_video_metadata.to_csv(filename + '_metadata.csv')
    df_video_rel.to_csv(filename + '_relations.csv')
    return df_video_metadata, df_video_rel
    

def load_channel_data(filename, replace_node_ids=True):
    df_metadata, df_rel = parse_gdf_channel(filename + '.gdf')        
    # clean column names and update node ids
    df_metadata, df_graph = join_update_clean_dfs(df_rel, df_metadata, filename, replace_node_ids=replace_node_ids)
    df_metadata['url'] = 'https://www.youtube.com/channel/' + df_metadata['nodeid']
    df_graph = df_graph.dropna()
    df_metadata.to_csv(filename + '_metadata.csv')
    df_graph.to_csv(filename + '_relations.csv')
    return df_metadata, df_graph
    

    
df_video_metadata, df_video_graph = load_video_data('masha_and_shark_2019_05_14')